The goal of this code is to take un-formated .nc data and output it in a readble .csv file. After running this code, the user will have a csv file of variables representing sea ice concentration each day for a specified year. To begin, download the Near Real Time CDR (ID G10016) data set from NSIDC. This can be accomplished with a variety of methods, but I personally used FileZilla to batch download the daily data, however the NSIDC website outlines a few other batch downloaders that can be used. The additional 'psn25area_v3.csv' csv file used in this code will need to be downloaded and saved to the same working directory you use in this code. 

Special thank you to Ann Windnagel for contributing to the construction of this code. 

To begin, ensure the path in this box is set for where your data was saved to. For me, I created a file on my desktop that contained all of the data for each year (seperately). In this code, I was reading in the 2019 dat set, so my path is through my user to desktop, into the 2019 folder, and in an additional sub-folder titled "2019". This information can be easily copy and pasted from your file directory when you access the location the data is stored in.

In [2]:
import glob
import os
import os.path as op
import csv
import numpy as np
import numpy.ma as ma
from netCDF4 import Dataset

# Change path to where data and code reside
HOME = op.join(op.expanduser('~'))
path = "C:/Users/misi7940/Desktop/2019/2019"
os.chdir(path)


In the following code, ensure you change the file name to what will be used in the full compiler code. As of current, the later code relies on the data for 2018 being saved as "index2018.csv" and the data for 2019 as "index2019.csv". These changes can be made in the line "with open('index2019.csv','w',newline='') as f1:". 

In [3]:
# Open CDR NetCDF file
#filename = "seaice_conc_daily_icdr_nh_f18_20180102_v01r00.nc"
files=(glob.glob('*.nc'))
files=files[0:365]

a=[]
with open('index2019.csv','w',newline='') as f1:
    writer=csv.writer(f1, delimiter=',')
    
    for f in files:
        dataset = Dataset(f)
        # Open csv file of pixel area size
        pixel_area_fn = 'psn25area_v3.csv'
        pixel_area = np.loadtxt(fname = pixel_area_fn, delimiter=",")
    
    # Scale the pixel area file by 1000
        pixel_area_scaled = pixel_area/1000
        seaice_conc = dataset.variables['seaice_conc_cdr']
        seaice_conc_np = np.array(seaice_conc)
    
        # convert seaice_conc_np to a 2d array so it's easier to work with
        seaice_conc_np_2d = np.copy(seaice_conc_np.squeeze())
        
        # Find all the spots where the sea ice conc is between 0 and 1 because I need to be able to scale just these data
        ans = np.where(np.logical_and(np.greater(seaice_conc_np_2d,0.0),np.less_equal(seaice_conc_np_2d,1.0)))
        
        # scale the concentration data by multipling the concentration by 100 to get percent
        seaice_conc_np_2d_percent = np.copy(seaice_conc_np_2d)
        seaice_conc_np_2d_percent[ans] = seaice_conc_np_2d_percent[ans]*100
        
        
        # In[5]:
        
        
        # Create mask to filter out the flag values from the computation and concentration equal to or less than 15%
        # Flag values are 252, 253, 254, 255; Note: not removing 251 because that's the pole hole and it is
        # included in the extent total
        # This line removes flag values and concentration <= 15%
        flag_mask = ma.masked_outside(seaice_conc_np_2d_percent, 15, 251)
        
        
        # In[6]:
        
        
        # Now need to set all non-masked values to 1
        non_mask = ma.nonzero(flag_mask)
        flag_mask[non_mask] = 1.0
        
        
        # In[8]:
        
        
        # Apply the pixel area array to the masked sea ice
        sea_ice_extent_raster = flag_mask * pixel_area_scaled
        cdr_seaice_exent_total = sea_ice_extent_raster.sum()
        #print(cdr_seaice_exent_total, 'Million sq km')
        
        row=[cdr_seaice_exent_total]
        writer.writerow(row)

After the code has run successfully, check the csv file to ensure it was created successfully. While checking the csv file, notice the date is not included in this file. I found it easier to manually change the csv file to have the date in the first column and extent in the second column (date,extent). This can most likely be coded for with this code, but that is to the discretion of the user to do. The naming practices in the output csv file will need to follow those of the compiling code which use variables 'yyyymmdd' and 'extent' as the two column names. Again, these can be renamed at the users discretion, but you will need to ensure all codes are updated to reflect any changes in naming.